#Directory, Libraries and Data

In [1]:
%cd /content/drive/MyDrive/Business Analyst course/Segmentation/RFM

/content/drive/MyDrive/Business Analyst course/Segmentation/RFM


In [2]:
#import libraries
import pandas as pd

In [3]:
#Data
df = pd.read_csv("customer_data.csv").dropna()
df.head()

,customer_id,revenue,most_recent_visit,number_of_orders,recency_days
0,22086,777,5/14/2006,9,232
1,2290,1555,9/8/2006,16,115
2,26377,336,11/19/2006,5,43
3,24650,1189,10/29/2006,12,64
4,12883,1229,12/9/2006,12,23


#Preparing Dataframe

In [4]:
#Preparing basket / monetary
df['monetary'] = df.revenue / df.number_of_orders
df.head(1)

,customer_id,revenue,most_recent_visit,number_of_orders,recency_days,monetary
0,22086,777,5/14/2006,9,232,86.333333


In [5]:
#Changing variables names
df.rename(columns = {'number_of_orders': 'frequency',
                     'recency_days': 'recency'}, inplace = True)
df.head(0)

,customer_id,revenue,most_recent_visit,frequency,recency,monetary


In [6]:
#Removing unwanted variables
df = df.drop(columns = ['most_recent_visit', 'revenue'])
df.head(1)

,customer_id,frequency,recency,monetary
0,22086,9,232,86.333333


In [7]:
#Create groups point system
df['F'] = pd.qcut(df['frequency'], q = 3, labels = range(1,4,1))
df['M'] = pd.qcut(df['monetary'], q = 3, labels = range(1,4,1))
df['R'] = pd.qcut(df['recency'], q = 3, labels = range(3,0,-1))
df.head()

,customer_id,frequency,recency,monetary,F,M,R
0,22086,9,232,86.333333,2,1,1
1,2290,16,115,97.187500,3,2,2
2,26377,5,43,67.200000,1,1,3
3,24650,12,64,99.083333,3,2,3
4,12883,12,23,102.416667,3,2,3


In [8]:
#Create RFM score
df['RFM'] = df[['R', 'F', 'M']].sum(axis = 1)
df.head(1)

,customer_id,frequency,recency,monetary,F,M,R,RFM
0,22086,9,232,86.333333,2,1,1,4


In [9]:
#Create RFM function
def rfm_segment(df):
  if df['RFM'] >= 8:
    return 'Superstar'
  elif ((df['RFM'] >= 5) and (df['RFM'] < 8)):
    return 'High Potential'
  else:
    return 'Low Relevance'

#RFM

In [10]:
#Applying function
df['RFM_level'] = df.apply(rfm_segment, axis = 1)
df.head()

,customer_id,frequency,recency,monetary,F,M,R,RFM,RFM_level
0,22086,9,232,86.333333,2,1,1,4,Low Relevance
1,2290,16,115,97.187500,3,2,2,7,High Potential
2,26377,5,43,67.200000,1,1,3,5,High Potential
3,24650,12,64,99.083333,3,2,3,8,Superstar
4,12883,12,23,102.416667,3,2,3,8,Superstar


In [11]:
#Aggregating the segments
df.groupby('RFM_level').agg({
    'recency': 'mean',
    'frequency': 'mean',
    'monetary': ['mean', 'count']
}).round(1)

recency frequency monetary       
                  mean      mean     mean  count
RFM_level                                       
High Potential   171.8       9.8     97.0  26445
Low Relevance    306.6       7.1     78.5   7179
Superstar         80.1      12.8    108.3   6375